In [1]:
import sys

sys.path.append('../')

In [2]:
import yaml

In [3]:
tic_list = yaml.load(open('ticlist.yml', 'r'))

/home/liuc/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """Entry point for launching an IPython kernel.


In [4]:
start_date = '2009-01-01'
end_date = '2021-08-30'

In [11]:
history_df_name = './hist_px.csv'

### Data Extraction 

In [5]:
from neo_finrl.data_processors.processor_yahoofinance import YahooFinanceProcessor

In [6]:
data_downloader = YahooFinanceProcessor()

In [9]:
stock_history_df = data_downloader.download_data(start_date, end_date, tic_list['tic'], '1D')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (25756, 9)


In [13]:
if history_df_name != None:
    stock_history_df.to_csv(history_df_name, index = False)

In [17]:
# simple hack for currency 
for col_i in ['open', 'high', 'low', 'close', 'adjcp']:
    stock_history_df.loc[stock_history_df.tic.str.endswith('.SI'), col_i] = \
    stock_history_df.loc[stock_history_df.tic.str.endswith('.SI'), col_i]/1.3

    stock_history_df.loc[stock_history_df.tic.str.endswith('.HK'), col_i] = \
    stock_history_df.loc[stock_history_df.tic.str.endswith('.HK'), col_i]/7.8

### Preprocess data

In [29]:
tech_indicators = ['macd',
 'rsi_30',
 'cci_30',
 'dx_30']

In [31]:
stock_data_df = data_downloader.clean_data(stock_history_df)
stock_data_df = data_downloader.add_technical_indicator(stock_data_df, tech_indicators)

Clean data for 2343.HK
Data clean for 2343.HK is finished.
Clean data for CHJ.SI
Data clean for CHJ.SI is finished.
Clean data for CPLP
Data clean for CPLP is finished.
Clean data for CTRM
NaN data on start date, fill using first valid data.
Data clean for CTRM is finished.
Clean data for DSX
Data clean for DSX is finished.
Clean data for ESEA
Data clean for ESEA is finished.
Clean data for GSL
Data clean for GSL is finished.
Clean data for SB
Data clean for SB is finished.
Clean data for SBLK
Data clean for SBLK is finished.
Data clean all finished!


/home/liuc/anaconda3/lib/python3.7/site-packages/stockstats.py:483: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  lambda x: np.fabs(x - x.mean()).mean())


In [32]:
stock_data_df.sample(5)

,time,open,high,low,close,adjcp,volume,tic,macd,rsi_30,cci_30,dx_30
16214,2009-07-10,340.000000,352.799988,328.799988,343.200012,257.564209,660.0,ESEA,-20.765676,43.398285,-114.412021,42.884684
10347,2011-07-18,2.250000,2.250000,2.250000,2.250000,2.250000,0.0,CTRM,0.000000,NaN,NaN,NaN
3294,2009-02-17,1.252235,1.252235,1.252235,1.252235,0.898850,3225.0,CHJ.SI,-0.010660,38.655702,-70.027783,12.752721
7938,2014-08-20,76.720001,77.000000,76.230003,76.860001,38.596691,24186.0,CPLP,-0.227961,49.660030,-59.177493,23.887443
24402,2016-09-20,1.480000,1.590000,1.450000,1.490000,1.490000,209500.0,SB,0.050426,57.567309,140.011505,40.702603


In [33]:
stock_data_df.to_csv('cleaned_stock.csv', index = False)

### To Do: create env and test RL